In [3]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.41


In [32]:
!conda install progressbar

Solving environment: - ^C
| 

In [11]:
import os
from azureml.core import Workspace, Experiment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.data.data_reference import DataReference
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

In [24]:
%%script sh --out /dev/null --err /dev/null
cd .. && mkdir -p data/mortgage_2000 && cd data/mortgage_2000 && wget http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data/mortgage_2000.tgz

In [25]:
%%sh 
cd ../data/mortgage_2000 && tar -xvf mortgage_2000.tgz

names.csv
acq/Acquisition_2000Q4.txt
acq/Acquisition_2000Q3.txt
acq/Acquisition_2000Q2.txt
acq/Acquisition_2000Q1.txt
perf/Performance_2000Q4.txt
perf/Performance_2000Q3.txt
perf/Performance_2000Q2.txt
perf/Performance_2000Q1.txt


In [13]:
subscription_id = os.environ.get("SUBSCRIPTION_ID", "")
resource_group = os.environ.get("RESOURCE_GROUP", "")
workspace_name = os.environ.get("WORKSPACE_NAME", "")
# workspace_region = os.environ.get("WORKSPACE_REGION", "")

ws = Workspace(workspace_name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

# write config to a local directory for future use
# ws.write_config()

In [40]:
scripts_folder = "scripts"

# import shutil
# shutil.copy('./1_pandasVsRapids_ETL.py', os.path.join(scripts_folder, '1_pandasVsRapids_ETL.py'))

# with open(os.path.join(scripts_folder, './1_pandasVsRapids_ETL.py'), 'r') as process_data_script:
#     print(process_data_script.read())

FileNotFoundError: [Errno 2] No such file or directory: './1_pandasVsRapids_ETL.py'

In [16]:
gpu_cluster_name = "gpu-todrabas"

if gpu_cluster_name in ws.compute_targets:
    gpu_cluster = ws.compute_targets[gpu_cluster_name]
    
    if gpu_cluster and type(gpu_cluster) is AmlCompute:
        print('found compute target. just use it. ' + gpu_cluster_name)
else:
    print("creating new cluster")
    # vm_size parameter below could be modified to one of the RAPIDS-supported VM types
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "Standard_NC6s_v2", min_nodes=1, max_nodes = 1)

    # create the cluster
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, provisioning_config)
    gpu_cluster.wait_for_completion(show_output=True)

creating new cluster
Creating
Succeeded........................
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [34]:
import tarfile
import hashlib
from urllib.request import urlretrieve
# from progressbar import ProgressBar

def validate_downloaded_data(path):
    if(os.path.isdir(path) and os.path.exists(path + '//names.csv')) :
        if(os.path.isdir(path + '//acq' ) and len(os.listdir(path + '//acq')) == 8):
            if(os.path.isdir(path + '//perf' ) and len(os.listdir(path + '//perf')) == 11):
                print("Data has been downloaded and decompressed at: {0}".format(path))
                return True
    print("Data has not been downloaded and decompressed")
    return False

# def show_progress(count, block_size, total_size):
#     global pbar
#     global processed
    
#     if count == 0:
#         pbar = ProgressBar(maxval=total_size)
#         processed = 0
    
#     processed += block_size
#     processed = min(processed,total_size)
#     pbar.update(processed)

        
def download_file(fileroot):
    filename = fileroot + '.tgz'
    if(not os.path.exists(filename) or hashlib.md5(open(filename, 'rb').read()).hexdigest() != '82dd47135053303e9526c2d5c43befd5' ):
        url_format = 'http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data/{0}.tgz'
        url = url_format.format(fileroot)
        print("...Downloading file :{0}".format(filename))
        urlretrieve(url, filename)
#         pbar.finish()
        print("...File :{0} finished downloading".format(filename))
    else:
        print("...File :{0} has been downloaded already".format(filename))
    return filename

def decompress_file(filename,path):
    tar = tarfile.open(filename)
    print("...Getting information from {0} about files to decompress".format(filename))
    members = tar.getmembers()
    numFiles = len(members)
    so_far = 0
    for member_info in members:
        tar.extract(member_info,path=path)
#         show_progress(so_far, 1, numFiles)
        so_far += 1
#     pbar.finish()
    print("...All {0} files have been decompressed".format(numFiles))
    tar.close()

In [35]:
fileroot = 'mortgage_2000'
path = '.\\{0}'.format(fileroot)
pbar = None
processed = 0

if(not validate_downloaded_data(path)):
    print("Downloading and Decompressing Input Data")
    filename = download_file(fileroot)
    decompress_file(filename,path)
    print("Input Data has been Downloaded and Decompressed")

Data has not been downloaded and decompressed
...Downloading file :mortgage_2000.tgz
...File :mortgage_2000.tgz finished downloading
...Getting information from mortgage_2000.tgz about files to decompress
...All 9 files have been decompressed
Input Data has been Downloaded and Decompressed


In [36]:
# fileroot = 'data\mortagage_2000'
# path = '.\\{0}'.format(fileroot)
# print(path)
ds = ws.get_default_datastore()

# download and uncompress data in a local directory before uploading to data store
# directory specified in src_dir parameter below should have the acq, perf directories with data and names.csv file
ds.upload(src_dir=path, target_path=fileroot, overwrite=True, show_progress=True)

# data already uploaded to the datastore
data_ref = DataReference(data_reference_name='data', datastore=ds, path_on_datastore=fileroot)

Uploading .\mortgage_2000/acq/Acquisition_2000Q1.txt
Uploading .\mortgage_2000/acq/Acquisition_2000Q2.txt
Uploading .\mortgage_2000/acq/Acquisition_2000Q3.txt
Uploading .\mortgage_2000/acq/Acquisition_2000Q4.txt
Uploading .\mortgage_2000/names.csv
Uploading .\mortgage_2000/perf/Performance_2000Q1.txt
Uploading .\mortgage_2000/perf/Performance_2000Q2.txt
Uploading .\mortgage_2000/perf/Performance_2000Q3.txt
Uploading .\mortgage_2000/perf/Performance_2000Q4.txt
Uploaded .\mortgage_2000/names.csv, 1 files out of an estimated total of 9
Uploaded .\mortgage_2000/acq/Acquisition_2000Q1.txt, 2 files out of an estimated total of 9
Uploaded .\mortgage_2000/acq/Acquisition_2000Q2.txt, 3 files out of an estimated total of 9
Uploaded .\mortgage_2000/acq/Acquisition_2000Q4.txt, 4 files out of an estimated total of 9
Uploaded .\mortgage_2000/acq/Acquisition_2000Q3.txt, 5 files out of an estimated total of 9
Uploaded .\mortgage_2000/perf/Performance_2000Q2.txt, 6 files out of an estimated total of 9


Fri May 31 18:24:56 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:03:00.0  On |                  N/A |
| 31%   46C    P8    29W / 250W |   2996MiB / 12034MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN V             Off  | 00000000:04:00.0 Off |                  N/A |
| 28%   38C    P8    N/A /  N/A |     12MiB / 12036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [38]:
run_config = RunConfiguration()
run_config.framework = 'python'
run_config.environment.python.user_managed_dependencies = True
run_config.environment.python.interpreter_path = '/conda/envs/rapids/bin/python'
run_config.target = gpu_cluster_name
run_config.environment.docker.enabled = True
run_config.environment.docker.gpu_support = True
run_config.environment.docker.base_image = "rapidsai/rapidsai:cuda9.2-runtime-ubuntu18.04"
# run_config.environment.docker.base_image_registry.address = '<registry_url>' # not required if the base_image is in Docker hub
# run_config.environment.docker.base_image_registry.username = '<user_name>' # needed only for private images
# run_config.environment.docker.base_image_registry.password = '<password>' # needed only for private images
run_config.environment.spark.precache_packages = False
run_config.data_references={'data':data_ref.to_config()}

In [ ]:
src = ScriptRunConfig(source_directory=scripts_folder, 
                          script='1_pandasVsRapids_ETL.py', 
                          arguments = ['--gpu', 0, '--data_dir', str(data_ref)
                                      ],
                          run_config=run_config
                         )

exp = Experiment(ws, 'rapidstest_cpu')
run = exp.submit(config=src)
# RunDetails(run).show()
run.wait_for_completion(show_output=True)

RunId: rapidstest_cpu_1559329102_76a4e0f3
Web View: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/MLADS_todrabas/providers/Microsoft.MachineLearningServices/workspaces/todrabas_MLADS_WE/experiments/rapidstest_cpu/runs/rapidstest_cpu_1559329102_76a4e0f3

Streaming azureml-logs/80_driver_log.txt

Running ETL...
/mnt/batch/tasks/shared/LS_root/jobs/todrabas_mlads_we/azureml/rapidstest_cpu_1559329102_76a4e0f3/mounts/workspaceblobstore/mortgage_2000/acq/Acquisition_2000Q1.txt
/mnt/batch/tasks/shared/LS_root/jobs/todrabas_mlads_we/azureml/rapidstest_cpu_1559329102_76a4e0f3/mounts/workspaceblobstore/mortgage_2000/perf/Performance_2000Q1.txt


In [ ]:

# delete the cluster
# gpu_cluster.delete()